In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
folder_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\qml.Sample\OptimiseParams\strategy"
potential = 'DW'
cutoff = 16

dataframes = {}

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
time_taken = []

num_vqe = []
shots = []
tolerance = []
abs_tolerance = []
strategy = []
popsize = []
maxiter = []


for f in folders:

    data_path = folder_path + "\\{}\\{}_{}.json".format(f,potential, cutoff)
    
    with open(data_path, 'r') as file:
        data = json.load(file)

    num_vqe.append(data['num_VQE'])
    shots.append(data['shots'])

    optimzer = data['Optimizer']
    maxiter.append(optimzer['maxiter'])
    tolerance.append(optimzer['tolerance'])
    abs_tolerance.append(optimzer['abs_tolerance'])
    strategy.append(optimzer['strategy'])
    popsize.append(optimzer['popsize'])

    success = sum(data['success'])
    
    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)

    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))
    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))
    median_e.append(np.median(data['results']))
    delta_median_e.append(abs(exact_min_e - np.median(data['results'])))
    
    time_taken.append(data['total_VQE_time'])

    

In [3]:
df = pd.DataFrame({
    'Strategy': folders,
    'num_vqe': num_vqe,
    'shots': shots,
    'tolerance': tolerance,
    'abs_tolerance': abs_tolerance,
    'strategy': strategy,
    'popsize': popsize,
    'Converged Runs': [f"{c}/40" for c in converged_runs],
    'maxiter': maxiter,
    'Mean Iter': mean_iters,
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'Time': time_taken
})

In [6]:
df.sort_values('Delta median E', ascending=True)

,Strategy,tolerance,abs_tolerance,popsize,Converged Runs,Mean Iter,Delta min E,Delta median E,Exact,Time
1,best1exp,1.000000e-08,1.000000e-07,5,100/40,144,3.01372e+00,1.01139e+00,8.91599e-01,2:02:11.681679
4,currenttobest1bin,1.000000e-08,1.000000e-07,5,100/40,454,2.84895e+00,1.04099e+00,8.91599e-01,5:50:28.400607
9,randtobest1exp,1.000000e-08,1.000000e-07,5,100/40,202,2.94345e+00,1.15358e+00,8.91599e-01,2:50:56.512390
6,rand1bin,1.000000e-08,1.000000e-07,5,100/40,378,2.68476e+00,1.34881e+00,8.91599e-01,5:05:07.830955
5,currenttobest1exp,1.000000e-08,1.000000e-07,5,98/40,966,3.04986e+00,1.40120e+00,8.91599e-01,12:12:50.280389
7,rand1exp,1.000000e-08,1.000000e-07,5,92/40,1465,3.14985e+00,1.60566e+00,8.91599e-01,17:46:46.830114
2,best2bin,1.000000e-08,1.000000e-07,5,3/40,9736,3.36666e+00,1.73023e+00,8.91599e-01,"5 days, 13:53:19.420281"
3,best2exp,1.000000e-08,1.000000e-07,5,1/40,9907,3.12823e+00,1.81447e+00,8.91599e-01,"5 days, 22:58:36.218124"
0,best1bin,1.000000e-08,1.000000e-07,5,100/40,88,2.42916e+00,5.30390e-01,8.91599e-01,1:11:51.840134
8,randtobest1bin,1.000000e-08,1.000000e-07,5,100/40,114,2.64452e+00,7.77555e-01,8.91599e-01,1:38:20.828214


In [5]:
from tabulate import tabulate
df = df.drop(columns=['strategy','num_vqe','shots','maxiter']).sort_values('Mean Iter',ascending=True)

latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex")#, floatfmt=".3f")
print(latex_table)


\begin{tabular}{lrrrlrrrrl}
\hline
 Strategy          &   tolerance &   abs\_tolerance &   popsize & Converged Runs   &   Mean Iter &   Delta min E &   Delta median E &    Exact & Time                    \\
\hline
 best1bin          &       1e-08 &           1e-07 &         5 & 100/40           &          88 &       2.42916 &         0.53039  & 0.891599 & 1:11:51.840134          \\
 randtobest1bin    &       1e-08 &           1e-07 &         5 & 100/40           &         114 &       2.64452 &         0.777555 & 0.891599 & 1:38:20.828214          \\
 best1exp          &       1e-08 &           1e-07 &         5 & 100/40           &         144 &       3.01372 &         1.01139  & 0.891599 & 2:02:11.681679          \\
 randtobest1exp    &       1e-08 &           1e-07 &         5 & 100/40           &         202 &       2.94345 &         1.15358  & 0.891599 & 2:50:56.512390          \\
 rand1bin          &       1e-08 &           1e-07 &         5 & 100/40           &         378 &     